In [147]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [148]:
df = pd.read_csv("../../../data/superstore.csv", encoding="latin-1", parse_dates=["Order Date", "Ship Date"])
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9994 entries, 0 to 9993
Data columns (total 21 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   Row ID         9994 non-null   int64         
 1   Order ID       9994 non-null   object        
 2   Order Date     9994 non-null   datetime64[ns]
 3   Ship Date      9994 non-null   datetime64[ns]
 4   Ship Mode      9994 non-null   object        
 5   Customer ID    9994 non-null   object        
 6   Customer Name  9994 non-null   object        
 7   Segment        9994 non-null   object        
 8   Country        9994 non-null   object        
 9   City           9994 non-null   object        
 10  State          9994 non-null   object        
 11  Postal Code    9994 non-null   int64         
 12  Region         9994 non-null   object        
 13  Product ID     9994 non-null   object        
 14  Category       9994 non-null   object        
 15  Sub-Category   9994 n

In [149]:
#Practicas con el agg
df["Dias_Envio"] = (df["Ship Date"] - df["Order Date"]).dt.days
resumen = df.groupby("Category").agg({
    "Dias_Envio": "mean",
    "Sales": "sum",
    "Profit": ["sum","mean"],
    "Discount": "max"
})
resumen

Dias_Envio        Sales       Profit            Discount
                      mean          sum          sum       mean      max
Category                                                                
Furniture         3.917020  741999.7953   18451.2728   8.699327      0.7
Office Supplies   3.983405  719047.0320  122490.8008  20.327050      0.8
Technology        3.923119  836154.0330  145454.9481  78.752002      0.7

In [150]:
#Practicas con la tabla pivot, asi sacamos en q region es mas rentable la categoria segun el profit
#Un cruado comparativo se podria decir
matriz_profit = df.pivot_table(index="Category", columns="Region",values="Profit")
matriz_profit

Region,Central,East,South,West
Category,,,,
Furniture,-5.968918,5.068496,20.395199,16.272914
Office Supplies,6.244712,23.957114,20.086827,27.733183
Technology,80.231981,88.714084,68.231506,73.962687


In [151]:
"""# Es como el Excel: Filas (index), Columnas (columns) y Valores (values)
tabla_dinamica = df.pivot_table(
    index='Region', 
    columns='Segment', 
    values='Profit', 
    aggfunc='sum' # Aquí puedes poner 'mean', 'sum', 'count'
)

print(tabla_dinamica)"""

"# Es como el Excel: Filas (index), Columnas (columns) y Valores (values)\ntabla_dinamica = df.pivot_table(\n    index='Region', \n    columns='Segment', \n    values='Profit', \n    aggfunc='sum' # Aquí puedes poner 'mean', 'sum', 'count'\n)\n\nprint(tabla_dinamica)"

In [ ]:
def evaluar_envio(fila):
    dias = fila['Dias_Envio']
    modo = fila['Ship Mode']
    
    if modo == 'Same Day' and dias > 0: return 'Retrasado'
    if modo == 'First Class' and dias > 2: return 'Lento 1'
    if modo == "Second Class" and dias > 3: return "Lento 2"
    if modo == "Standard Class" and dias > 4: return "Lento 3"
    if dias > 5: return 'Crítico'
    return 'A tiempo'

# Aplicamos la función a todo el DataFrame (axis=1 para leer filas, si es 0 como predeterminado agarra la columna entera)
# tambien se puede hacer funcion lambda para comparar una columna con otra y te crea un resultado 
df['Status_Envio'] = df.apply(evaluar_envio, axis=1)
print(df['Status_Envio'].value_counts())

Status_Envio
A tiempo     4982
Lento 3      3564
Lento 2       800
Lento 1       624
Retrasado      24
Name: count, dtype: int64


In [153]:
# Practica con marge para unir dataframe (o tabla) con otro dataFrame

data_gerentes = {
    "Region": ["Central","East","South","West"],
    "Gerente": ["Elias B", "Renzo A", "Sol M", "Nicolas P"]
}

df_gerentes = pd.DataFrame(data_gerentes)
df = df.merge(df_gerentes,how="left",on="Region")
df[["Region","Gerente"]].head()

,Region,Gerente
0,South,Sol M
1,South,Sol M
2,West,Nicolas P
3,South,Sol M
4,South,Sol M


In [154]:
# Practica con transform, para comparar una fila contra 
# 1. Calculamos el total de ventas de CADA categoría y lo ponemos en cada fila
df['Ventas_Totales_Cat'] = df.groupby('Category')['Sales'].transform('sum')

# 2. Ahora podemos saber qué porcentaje representa ESTA venta sobre el total de su categoría
df['Ratio_Sobre_Categoria'] = (df['Sales'] / df['Ventas_Totales_Cat']) * 100

df[['Category', 'Sales', 'Ventas_Totales_Cat', 'Ratio_Sobre_Categoria']].head()

,Category,Sales,Ventas_Totales_Cat,Ratio_Sobre_Categoria
0,Furniture,261.9600,741999.7953,0.035305
1,Furniture,731.9400,741999.7953,0.098644
2,Office Supplies,14.6200,719047.0320,0.002033
3,Furniture,957.5775,741999.7953,0.129054
4,Office Supplies,22.3680,719047.0320,0.003111


## Practicar con esto:

Usa un pivot_table para ver el Margen % promedio (el que corregimos ayer) cruzando Segment y Ship Mode. ¿Hay algún modo de envío que sea especialmente poco rentable para el segmento Consumer?

Crea una columna con apply() que clasifique las ventas en "Pequeña", "Mediana" o "Corporativa" (ej: más de $500 es corporativa).

Análisis de Envíos: Ya que tienes Dias_Envio, usa .describe() sobre esa columna para ver el promedio general. ¿Cuál es el máximo? ¿Hay algún error de datos (días negativos)?